<a href="https://colab.research.google.com/github/semih108/car-price-prediction/blob/main/scraping_car_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Audi scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

base_url = "https://www.autoscout24.at/lst/audi"

audi_models = [
    {'make': 'audi', 'model': 'a5'},
    {'make': 'audi', 'model': '100'},
    {'make': 'audi', 'model': '200'},
    {'make': 'audi', 'model': '50'},
    {'make': 'audi', 'model': '80'},
    {'make': 'audi', 'model': '90'},
    {'make': 'audi', 'model': 'a1'},
    {'make': 'audi', 'model': 'a2'},
    {'make': 'audi', 'model': 'a3'},
    {'make': 'audi', 'model': 'a4'},
    {'make': 'audi', 'model': 'a4-allroad'},
    {'make': 'audi', 'model': 'a5'},
    {'make': 'audi', 'model': 'a6'},
    {'make': 'audi', 'model': 'a6-allroad'},
    {'make': 'audi', 'model': 'a7'},
    {'make': 'audi', 'model': 'a8'},
    {'make': 'audi', 'model': 'allroad'},
    {'make': 'audi', 'model': 'cabriolet'},
    {'make': 'audi', 'model': 'coupe'},
    {'make': 'audi', 'model': 'e-tron'},
    {'make': 'audi', 'model': 'e-tron-gt'},
    {'make': 'audi', 'model': 'q1'},
    {'make': 'audi', 'model': 'q2'},
    {'make': 'audi', 'model': 'q3'},
    {'make': 'audi', 'model': 'q4-e-tron'},
    {'make': 'audi', 'model': 'q5'},
    {'make': 'audi', 'model': 'q6'},
    {'make': 'audi', 'model': 'q7'},
    {'make': 'audi', 'model': 'q8'},
    {'make': 'audi', 'model': 'q8-e-tron'},
    {'make': 'audi', 'model': 'quattro'},
    {'make': 'audi', 'model': 'r8'},
    {'make': 'audi', 'model': 'rs'},
    {'make': 'audi', 'model': 'rs-e-tron-gt'},
    {'make': 'audi', 'model': 'rs-q3'},
    {'make': 'audi', 'model': 'rs-q5'},
    {'make': 'audi', 'model': 'rs-q8'},
    {'make': 'audi', 'model': 'rs2'},
    {'make': 'audi', 'model': 'rs3'},
    {'make': 'audi', 'model': 'rs4'},
    {'make': 'audi', 'model': 'rs5'},
    {'make': 'audi', 'model': 'rs6'},
    {'make': 'audi', 'model': 'rs7'},
    {'make': 'audi', 'model': 's1'},
    {'make': 'audi', 'model': 's2'},
    {'make': 'audi', 'model': 's3'},
    {'make': 'audi', 'model': 's4'},
    {'make': 'audi', 'model': 's5'},
    {'make': 'audi', 'model': 's6'},
    {'make': 'audi', 'model': 's7'},
    {'make': 'audi', 'model': 's8'},
    {'make': 'audi', 'model': 'sonstige'},
    {'make': 'audi', 'model': 'sq2'},
    {'make': 'audi', 'model': 'sq3'},
    {'make': 'audi', 'model': 'sq5'},
    {'make': 'audi', 'model': 'sq6'},
    {'make': 'audi', 'model': 'sq7'},
    {'make': 'audi', 'model': 'sq8'},
    {'make': 'audi', 'model': 'sq8-e-tron'},
    {'make': 'audi', 'model': 'tt'},
    {'make': 'audi', 'model': 'tt-rs'},
    {'make': 'audi', 'model': 'tts'},
    {'make': 'audi', 'model': 'v8'}
]

def get_page_data(soup, data_list, make, model):
    for car in soup.find_all('div', class_='ListItem_wrapper__TxHWu'):
        try:
            name = car.find('h2').get_text(strip=True)
        except AttributeError:
            name = "N/A"

        try:
            price = car.find('p', class_='Price_price__APlgs').get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            details = car.find('div', class_='VehicleDetailTable_container__XhfV1')
            mileage = details.find('span', {'data-testid': 'VehicleDetails-mileage_road'}).get_text(strip=True)
            transmission = details.find('span', {'data-testid': 'VehicleDetails-transmission'}).get_text(strip=True)
            date = details.find('span', {'data-testid': 'VehicleDetails-calendar'}).get_text(strip=True)
            fuel = details.find('span', {'data-testid': 'VehicleDetails-gas_pump'}).get_text(strip=True)
            power = details.find('span', {'data-testid': 'VehicleDetails-speedometer'}).get_text(strip=True)
        except AttributeError:
            mileage = transmission = date = fuel = power = "N/A"

        data_list.append([make, model, name, price, mileage, transmission, date, fuel, power])

def scrape_autoscout24(make, model):
    data_list = []
    params = {
        'atype': 'C',
        'page': 1,
        'make': make,
        'model': model,
        'cy': 'A',
        'damaged_listing': 'exclude',
        'desc': '0',
        'powertype': 'kw',
        'sort': 'standard',
        'ustate': 'N,U'
    }

    while true:
        response = requests.get(f"{base_url}/{model}", params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        get_page_data(soup, data_list, make, model)
        next_button = soup.find('button', {'aria-label': 'Zu nächsten Seite'})
        if next_button and 'disabled' not in next_button.attrs.get('class', []):
            print(f"Next button found on page {params['page']}")
            params['page'] += 1
            time.sleep(2)
        else:
            print("No next button found or it is disabled.")
            break

    return data_list

all_data = [["Make", "Model", "Name", "Price", "Mileage", "Transmission", "Date", "Fuel", "Power"]]

for car_model in audi_models:
    data = scrape_autoscout24(car_model['make'], car_model['model'])
    all_data.extend(data)

# Save the data to a CSV file
with open('audi_car_prices_test.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(all_data)


# BMW scraping


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

base_url = "https://www.autoscout24.at/lst/bmw"

bmw_modelle = [
    "114", "116", "118", "120", "123", "125", "128", "130", "135", "140",
    "2002",
    "214", "216", "218", "220", "223", "225", "228", "230", "235", "240",
    "315", "316", "318", "320", "323", "324", "325", "328", "330", "335", "340", "Active Hybrid 3",
    "418", "420", "425", "428", "430", "435", "440",
    "518", "520", "523", "524", "525", "528", "530", "535", "540", "545", "550", "Active Hybrid 5",
    "620", "628", "630", "633", "635", "640", "645", "650",
    "725", "728", "730", "732", "735", "740", "745", "750", "760", "Active Hybrid 7",
    "830", "840", "850",
    "i3", "i4", "i5", "i7", "i8", "iX", "iX1", "iX2", "iX3",
    "1er M Coupé", "M1", "M2", "M3", "M4", "M5", "M550", "M6", "M8", "M850",
    "Active Hybrid X6", "X1", "X2", "X2 M", "X3", "X3 M", "X4", "X4 M", "X5", "X5 M", "X6", "X6 M", "X7", "X7 M", "XM",
    "Z1", "Z3", "Z3 M", "Z4", "Z4 M", "Z8",
    "Sonstige"
]

def get_page_data(soup, data_list, make, model):
    for car in soup.find_all('div', class_='ListItem_wrapper__TxHWu'):
        try:
            name = car.find('h2').get_text(strip=True)
        except AttributeError:
            name = "N/A"

        try:
            price = car.find('p', class_='Price_price__APlgs').get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            details = car.find('div', class_='VehicleDetailTable_container__XhfV1')
            mileage = details.find('span', {'data-testid': 'VehicleDetails-mileage_road'}).get_text(strip=True)
            transmission = details.find('span', {'data-testid': 'VehicleDetails-transmission'}).get_text(strip=True)
            date = details.find('span', {'data-testid': 'VehicleDetails-calendar'}).get_text(strip=True)
            fuel = details.find('span', {'data-testid': 'VehicleDetails-gas_pump'}).get_text(strip=True)
            power = details.find('span', {'data-testid': 'VehicleDetails-speedometer'}).get_text(strip=True)
        except AttributeError:
            mileage = transmission = date = fuel = power = "N/A"

        data_list.append([make, model, name, price, mileage, transmission, date, fuel, power])

def scrape_autoscout24(make, model):
    data_list = []
    params = {
        'atype': 'C',
        'page': 1,
        'make': make,
        'model': model,
        'cy': 'A',
        'damaged_listing': 'exclude',
        'desc': '0',
        'powertype': 'kw',
        'sort': 'standard',
        'ustate': 'N,U'
    }
    while True:
        response = requests.get(f"{base_url}/{model}", params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        get_page_data(soup, data_list, make, model)
        next_button = soup.find('button', {'aria-label': 'Zu nächsten Seite'})
        if next_button and 'disabled' not in next_button.attrs.get('class', []):
            params['page'] += 1
            print(f"Next button found on page {params['page']}")
            time.sleep(2)
        else:
            print("No next button found or it is disabled.")
            break

    return data_list

all_data = [["Make", "Model", "Name", "Price", "Mileage", "Transmission", "Date", "Fuel", "Power"]]

for model in bmw_modelle:
    data = scrape_autoscout24('bmw', model)
    all_data.extend(data)

# Save the data to a CSV file
with open('bmw_car_prices.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(all_data)


Next button found on page 2
Next button found on page 3
No next button found or it is disabled.
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
Next button found on page 10
Next button found on page 11
Next button found on page 12
Next button found on page 13
Next button found on page 14
Next button found on page 15
Next button found on page 16
Next button found on page 17
Next button found on page 18
No next button found or it is disabled.
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
Next button found on page 10
Next button found on page 11
Next button found on page 12
Next button found on page 13
Next button found on page 14
Next butto

# Mercedes scraping

In [ ]:
mercedes_modelle = [
    "A 180", "A 200", "A 220", "A 250", "A 35 AMG", "A 45 AMG",
    "B 180", "B 200", "B 220", "B 250", "B Electric Drive",
    "C 180", "C 200", "C 220", "C 250", "C 300", "C 350", "C 43 AMG", "C 63 AMG",
    "CLA 180", "CLA 200", "CLA 220", "CLA 250", "CLA 35 AMG", "CLA 45 AMG",
    "CLS 350", "CLS 450", "CLS 53 AMG", "CLS 63 AMG",
    "E 200", "E 220", "E 250", "E 300", "E 350", "E 400", "E 450", "E 53 AMG", "E 63 AMG",
    "G 350", "G 400", "G 500", "G 63 AMG", "G 65 AMG",
    "GLA 180", "GLA 200", "GLA 220", "GLA 250", "GLA 35 AMG", "GLA 45 AMG",
    "GLB 200", "GLB 220", "GLB 250", "GLB 35 AMG",
    "GLC 200", "GLC 220", "GLC 250", "GLC 300", "GLC 43 AMG", "GLC 63 AMG",
    "GLE 250", "GLE 300", "GLE 350", "GLE 400", "GLE 450", "GLE 500", "GLE 53 AMG", "GLE 63 AMG",
    "GLS 350", "GLS 400", "GLS 450", "GLS 500", "GLS 580", "GLS 63 AMG", "Maybach GLS",
    "S 350", "S 400", "S 450", "S 500", "S 560", "S 580", "S 600", "S 63 AMG", "S 65 AMG", "Maybach S-Klasse",
    "SL 400", "SL 450", "SL 500", "SL 550", "SL 63 AMG", "SL 65 AMG",
    "SLC 180", "SLC 200", "SLC 300", "SLC 43 AMG",
    "V 200", "V 220", "V 250",
    "EQA", "EQB", "EQC", "EQE", "EQS", "EQV",
    "CLA 35 AMG", "CLA 45 AMG", "CLC", "CLE", "CLE 200", "CLE 220", "CLE 300", "CLE 450", "CLE 530 AMG",
    "CLK 200", "CLK 220", "CLK 230", "CLK 240", "CLK 270", "CLK 280", "CLK 320", "CLK 350", "CLK 430", "CLK 500", "CLK 550 AMG", "CLK 630 AMG",
    "CLS 220", "CLS 250", "CLS 280", "CLS 300", "CLS 320", "CLS 400", "CLS 450", "CLS 500", "CLS 530 AMG", "CLS 550 AMG", "CLS 630 AMG",
    "E 230", "E 240", "E 260", "E 270", "E 280", "E 290", "E 320", "E 360 AMG", "E 420", "E 430 AMG", "E 430", "E 500 AMG", "E 530 AMG", "E 550 AMG", "E 550", "E 600 AMG", "E 630 AMG",
    "EQ-A 250", "EQ-A 300", "EQ-A 350", "EQ-B 250", "EQ-B 300", "EQ-B 350", "EQ-C 400", "EQ-E 300", "EQ-E 350", "EQ-E 430", "EQ-E 500", "EQ-E 530", "EQ-E SUV", "EQ-S SUV",
    "EQ-T", "EQ-V 250", "EQ-V 300", "G 230", "G 240", "G 250", "G 270", "G 280", "G 290", "G 320", "G 450", "G 550 AMG", "G 580", "G 630 AMG", "G 650 AMG",
    "GL 320", "GL 350", "GL 400", "GL 420", "GL 450", "GL 500", "GL 550 AMG", "GL 630 AMG",
    "GLA 350 AMG", "GLA 450 AMG", "GLB 180", "GLB 350 AMG", "GLC 350", "GLC 400", "GLC 430 AMG", "GLC 450", "GLC 630 AMG",
    "GLE 430 AMG", "GLE 530 AMG", "GLE 580", "GLE 630 AMG", "GLK 200", "GLK 220", "GLK 250", "GLK 280", "GLK 300", "GLK 320", "GLK 350",
    "GLS 580", "GLS 600", "GLS 630 AMG", "ML 230", "ML 250", "ML 270", "ML 280", "ML 300", "ML 320", "ML 350", "ML 400", "ML 420", "ML 430", "ML 450", "ML 500", "ML 550 AMG", "ML 630 AMG",
    "Marco Polo", "MB 100", "R 280", "R 300", "R 320", "R 350", "R 500", "R 630 AMG", "S 250", "S 260", "S 280", "S 300", "S 320"
]


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

base_url = "https://www.autoscout24.at/lst/mercedes-benz"

def get_page_data(soup, data_list, make, model):
    for car in soup.find_all('div', class_='ListItem_wrapper__TxHWu'):
        try:
            name = car.find('h2').get_text(strip=True)
        except AttributeError:
            name = "N/A"

        try:
            price = car.find('p', class_='Price_price__APlgs').get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            details = car.find('div', class_='VehicleDetailTable_container__XhfV1')
            mileage = details.find('span', {'data-testid': 'VehicleDetails-mileage_road'}).get_text(strip=True)
            transmission = details.find('span', {'data-testid': 'VehicleDetails-transmission'}).get_text(strip=True)
            date = details.find('span', {'data-testid': 'VehicleDetails-calendar'}).get_text(strip=True)
            fuel = details.find('span', {'data-testid': 'VehicleDetails-gas_pump'}).get_text(strip=True)
            power = details.find('span', {'data-testid': 'VehicleDetails-speedometer'}).get_text(strip=True)
        except AttributeError:
            mileage = transmission = date = fuel = power = "N/A"

        data_list.append([make, model, name, price, mileage, transmission, date, fuel, power])

def scrape_autoscout24(make, model):
    data_list = []
    params = {
        'atype': 'C',
        'page': 1,
        'make': make,
        'model': model,
        'cy': 'A',
        'damaged_listing': 'exclude',
        'desc': '0',
        'powertype': 'kw',
        'sort': 'standard',
        'ustate': 'N,U'
    }
    while True:
        response = requests.get(f"{base_url}/{model}", params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        get_page_data(soup, data_list, make, model)
        next_button = soup.find('button', {'aria-label': 'Zu nächsten Seite'})
        if next_button and 'disabled' not in next_button.attrs.get('class', []):
            params['page'] += 1
            print(f"Next button found on page {params['page']}")
            time.sleep(2)
        else:
            print("No next button found or it is disabled.")
            break

    return data_list

all_data = [["Make", "Model", "Name", "Price", "Mileage", "Transmission", "Date", "Fuel", "Power"]]

for model in mercedes_modelle:
    data = scrape_autoscout24('mercedes-benz', model)
    all_data.extend(data)

# Save the data to a CSV file
with open('mercedes-benz_car_prices.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(all_data)


Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
Next button found on page 10
Next button found on page 11
Next button found on page 12
Next button found on page 13
Next button found on page 14
Next button found on page 15
Next button found on page 16
Next button found on page 17
Next button found on page 18
Next button found on page 19
Next button found on page 20
Next button found on page 21
No next button found or it is disabled.
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
No next button found or it is disabled.
Next button found on page 2
Next button found on page 3
No next button found or it is disabled.
Next button found on page 2
Next button found on page 3


# VW scraping


In [ ]:
vw_modelle = [
    "181", "Amarok", "Anfibio", "Arteon", "Atlas", "Beetle", "Bora", "Buggy", "Bus", "Caddy", "CC", "Coccinelle",
    "Corrado", "Crafter", "Cross Touran", "Derby", "e-up!", "Eos", "Escarabajo", "Fox",
    "Cross Golf", "e-Golf", "Golf", "Golf Cabriolet", "Golf GTD", "Golf GTE", "Golf GTI", "Golf Plus", "Golf R",
    "Golf Sportsvan", "Golf Variant", "Grand California",
    "ID. Buzz", "ID. Buzz Cargo", "ID.3", "ID.4", "ID.5", "ID.6", "ID.7", "Iltis", "Jetta", "Käfer", "Karmann Ghia",
    "Kever", "L80", "LT", "Lupo", "Maggiolino", "New Beetle",
    "Passat", "Passat Alltrack", "Passat CC", "Passat Variant", "Phaeton", "Pointer",
    "Polo", "Polo Cross", "Polo GTI", "Polo Plus", "Polo R WRC", "Polo Sedan", "Polo Variant",
    "Routan", "Santana", "Scirocco", "Sharan", "T-Cross", "T-Roc", "T1", "T2",
    "T3", "T3 Blue Star", "T3 California", "T3 Caravelle", "T3 Kombi", "T3 Multivan", "T3 White Star",
    "T4", "T4 Allstar", "T4 California", "T4 Caravelle", "T4 Kombi", "T4 Multivan",
    "T5", "T5 California", "T5 Caravelle", "T5 Kombi", "T5 Multivan", "T5 Shuttle", "T5 Transporter",
    "T6 California", "T6 Caravelle", "T6 Kombi", "T6 Multivan", "T6 Transporter",
    "T6.1 California", "T6.1 Caravelle", "T6.1 Kombi", "T6.1 Multivan", "T6.1 Transporter",
    "T7 California", "T7 Kastenwagen", "T7 Multivan", "Taigo", "Taro",
    "Tiguan", "Tiguan Allspace", "Touareg", "Touran", "Transporter", "up!", "Vento", "Viloran", "XL1", "Sonstige"
]


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

base_url = "https://www.autoscout24.at/lst/volkswagen"

def get_page_data(soup, data_list, make, model):
    for car in soup.find_all('div', class_='ListItem_wrapper__TxHWu'):
        try:
            name = car.find('h2').get_text(strip=True)
        except AttributeError:
            name = "N/A"

        try:
            price = car.find('p', class_='Price_price__APlgs').get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            details = car.find('div', class_='VehicleDetailTable_container__XhfV1')
            mileage = details.find('span', {'data-testid': 'VehicleDetails-mileage_road'}).get_text(strip=True)
            transmission = details.find('span', {'data-testid': 'VehicleDetails-transmission'}).get_text(strip=True)
            date = details.find('span', {'data-testid': 'VehicleDetails-calendar'}).get_text(strip=True)
            fuel = details.find('span', {'data-testid': 'VehicleDetails-gas_pump'}).get_text(strip=True)
            power = details.find('span', {'data-testid': 'VehicleDetails-speedometer'}).get_text(strip=True)
        except AttributeError:
            mileage = transmission = date = fuel = power = "N/A"

        data_list.append([make, model, name, price, mileage, transmission, date, fuel, power])

def scrape_autoscout24(make, model):
    data_list = []
    params = {
        'atype': 'C',
        'page': 1,
        'make': make,
        'model': model,
        'cy': 'A',
        'damaged_listing': 'exclude',
        'desc': '0',
        'powertype': 'kw',
        'sort': 'standard',
        'ustate': 'N,U'
    }
    while True:
        response = requests.get(f"{base_url}/{model}", params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        get_page_data(soup, data_list, make, model)
        next_button = soup.find('button', {'aria-label': 'Zu nächsten Seite'})
        if next_button and 'disabled' not in next_button.attrs.get('class', []):
            print(f"Next button found on page {params['page']}")
            params['page'] += 1
            time.sleep(2)
        else:
            print("No next button found or it is disabled.")
            break

    return data_list

all_data = [["Make", "Model", "Name", "Price", "Mileage", "Transmission", "Date", "Fuel", "Power"]]

for model in vw_modelle:
    data = scrape_autoscout24('volkswagen', model)
    all_data.extend(data)

# Save the data to a CSV file
with open('volkswagen_car_prices.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(all_data)


Next button found on page 1
No next button found or it is disabled.
Next button found on page 1
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
Next button found on page 10
Next button found on page 11
Next button found on page 12
Next button found on page 13
Next button found on page 14
Next button found on page 15
No next button found or it is disabled.
No next button found or it is disabled.
Next button found on page 1
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
No next button found or it is disabled.
No next button found or it is disabled.
Next button found on page 1
Next button found on page 2
Next button found on page 3
Next butto

# skoda scraping


In [ ]:
skoda_modelle = [
    "105", "120", "130", "135", "Citigo", "Enyaq", "Fabia", "Favorit", "Felicia", "Forman",
    "Kamiq", "Karoq", "Kodiaq", "Octavia", "Pick-up", "Praktik", "Rapid/Spaceback", "Roomster",
    "Scala", "Snowman", "Superb", "Yeti", "Sonstige"
]


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

base_url = "https://www.autoscout24.at/lst/skoda"

def get_page_data(soup, data_list, make, model):
    for car in soup.find_all('div', class_='ListItem_wrapper__TxHWu'):
        try:
            name = car.find('h2').get_text(strip=True)
        except AttributeError:
            name = "N/A"

        try:
            price = car.find('p', class_='Price_price__APlgs').get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            details = car.find('div', class_='VehicleDetailTable_container__XhfV1')
            mileage = details.find('span', {'data-testid': 'VehicleDetails-mileage_road'}).get_text(strip=True)
            transmission = details.find('span', {'data-testid': 'VehicleDetails-transmission'}).get_text(strip=True)
            date = details.find('span', {'data-testid': 'VehicleDetails-calendar'}).get_text(strip=True)
            fuel = details.find('span', {'data-testid': 'VehicleDetails-gas_pump'}).get_text(strip=True)
            power = details.find('span', {'data-testid': 'VehicleDetails-speedometer'}).get_text(strip=True)
        except AttributeError:
            mileage = transmission = date = fuel = power = "N/A"

        data_list.append([make, model, name, price, mileage, transmission, date, fuel, power])

def scrape_autoscout24(make, model):
    data_list = []
    params = {
        'atype': 'C',
        'page': 1,
        'make': make,
        'model': model,
        'cy': 'A',
        'damaged_listing': 'exclude',
        'desc': '0',
        'powertype': 'kw',
        'sort': 'standard',
        'ustate': 'N,U'
    }
    while True:
        response = requests.get(f"{base_url}/{model}", params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        get_page_data(soup, data_list, make, model)
        next_button = soup.find('button', {'aria-label': 'Zu nächsten Seite'})
        if next_button and 'disabled' not in next_button.attrs.get('class', []):
            print(f"Next button found on page {params['page']}")
            params['page'] += 1
            time.sleep(2)
        else:
            print("No next button found or it is disabled.")
            break

    return data_list

all_data = [["Make", "Model", "Name", "Price", "Mileage", "Transmission", "Date", "Fuel", "Power"]]

for model in skoda_modelle:
    data = scrape_autoscout24('skoda', model)
    all_data.extend(data)

# Save the data to a CSV file
with open('skoda_car_prices.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(all_data)


No next button found or it is disabled.
Next button found on page 1
No next button found or it is disabled.
No next button found or it is disabled.
No next button found or it is disabled.
Next button found on page 1
Next button found on page 2
No next button found or it is disabled.
Next button found on page 1
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
Next button found on page 10
No next button found or it is disabled.
Next button found on page 1
Next button found on page 2
Next button found on page 3
Next button found on page 4
Next button found on page 5
Next button found on page 6
Next button found on page 7
Next button found on page 8
Next button found on page 9
Next button found on page 10
Next button found on page 11
Next button found on page 12
Next button found on page 13
Next button found on page 1